In [ ]:
#imports
import os
from owlready2 import get_ontology
from owlready2 import sync_reasoner_hermit
import traceback
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize

In [ ]:
#defining file path
directory = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
filename = 'oec-extracted.owl'
file_path = os.path.join(directory, filename)

try:
    # Loading the ontology
    onto = get_ontology("file://" + file_path).load()
    print("Ontology loaded successfully!")

except Exception as e:
    # if load fails
    print("Error occurred during ontology loading:")
    print(e)

#check for inconsistensies
inconsistencies = list(onto.inconsistent_classes())
if inconsistencies:
    print("Ontology is inconsistent!")
    print("Inconsistencies:")
    for inconsistency in inconsistencies:
        print(f"Inconsistency found involving: {inconsistency}")
else:
    print("Ontology is consistent.")    

In [ ]:
#first sync to check if reasoner works
try:
    with onto:
        sync_reasoner_hermit()
except Exception as e:
    print(f"Error: {e}")


In [ ]:
#checking inferred objects and its connections
inferred_properties = set()

for prop in onto.object_properties():
    try:
        for subj, obj in prop.get_relations():
            inferred_properties.add((prop, subj, obj))
    except Exception as e:
        print(f"An error occurred while processing object property '{prop.name}': {e}")

# Print inferred object properties and their usage
for prop, subj, obj in inferred_properties:
    try:
        subj_type = "Class" if subj in onto.classes() else "Individual" if subj in onto.individuals() else "Unknown"
        obj_type = "Class" if obj in onto.classes() else "Individual" if obj in onto.individuals() else "Unknown"

        print(f"Object Property '{prop.name}' inferred on {obj_type.lower()} '{obj.name}' because of '{subj.name}'")
    except Exception as e:
        print(f"An error occurred while formatting output: {e}")

In [ ]:
#  sparql queries
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX oec: <http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#>

    SELECT ?individual1 ?individual2
    WHERE {
        ?individual1 oec:termHasMeaningByContext ?individual2 .
    }
"""

try:
    # Executing SPARQL query
    results = onto.world.sparql(query)

    # Printing results
    for row in results:
        individual1 = row[0]
        individual2 = row[1]
        print(f"Individual connected through 'termHasMeaningByContext': {individual1.name} to {individual2.name}")

except Exception as e:
    print("Error occurred while executing the SPARQL query:")
    print(e)

In [ ]:
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX oec: <http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#>

    SELECT ?individual1 ?property ?individual2
    WHERE {
        ?individual1 ?property ?individual2 .
        FILTER (isIRI(?individual1) && isIRI(?individual2) && ?property != rdf:type)
    }
"""

try:
    results = onto.world.sparql(query)

    property_names = {
        "http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#actorHasOrganization": "Actor Has Organization",
    }

    for row in results:
        individual1 = row[0]
        property_uri = row[1]
        individual2 = row[2]
        
        property_name = property_names.get(str(property_uri), str(property_uri))
        
        print(f"Individual {individual1.name} is connected to {individual2.name} through property {property_name}")
        
except Exception as e:
    print("Error occurred while executing the SPARQL query:")
    print(e)


In [ ]:
# Printing inferred information for each class, property, and individual
print("Inferred Classes:")
for cls in onto.classes():
    cls_name = cls.name.split('#')[-1] if '#' in cls.name else cls.name
    print(f"\nClass: {cls_name}")
    instances = list(cls.instances())
    if instances:
        instance_names = [inst.name.split('#')[-1] if '#' in inst.name else inst.name for inst in instances]
        print(f"Inferred Instances: {instance_names}")
    else:
        print("No inferred instances.")

print("\nInferred Object Properties:")
for prop in onto.object_properties():
    prop_name = prop.name.split('#')[-1] if '#' in prop.name else prop.name
    print(f"\nObject Property: {prop_name}")
    relations = [(subj.name.split('#')[-1] if '#' in subj.name else subj.name,
                  obj.name.split('#')[-1] if '#' in obj.name else obj.name) for subj, obj in prop.get_relations()]
    if relations:
        for subj, obj in relations:
            print(f"Subject: {subj}, Object: {obj}")
    else:
        print("No inferred relations.")

print("\nInferred Data Properties:")
for data_prop in onto.data_properties():
    data_prop_name = data_prop.name.split('#')[-1] if '#' in data_prop.name else data_prop.name
    print(f"\nData Property: {data_prop_name}")
    values = [(subj.name.split('#')[-1] if '#' in subj.name else subj.name, value) for subj, value in data_prop.get_relations()]
    if values:
        for subj, value in values:
            print(f"Subject: {subj}, Value: {value}")
    else:
        print("No inferred values.")

print("\nInferred Individuals:")
for ind in onto.individuals():
    ind_name = ind.name.split('#')[-1] if '#' in ind.name else ind.name
    print(f"\nIndividual: {ind_name}")
    types = [typ.name.split('#')[-1] if '#' in typ.name else typ.name for typ in ind.is_a]
    if types:
        print(f"Inferred Types: {types}")
    else:
        print("No inferred types.")

In [ ]:
# Load the ontology using its IRI
onto = get_ontology("http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted").load()

# Retrieve individuals starting with "ExchangedTerm"
matching_individuals = [ind for ind in onto.individuals() if ind.name.startswith("ExchangedTerm")]

# Define the object property to focus on
target_property = onto.termHasMeaningByContext

# Iterate through the filtered individuals to find those with the target object property
for term_individual in matching_individuals:
    # Check if the individual has the target object property
    if hasattr(term_individual, "termHasMeaningByContext"):
        related_property = term_individual.termHasMeaningByContext
        if related_property:
            print(f"Individual '{term_individual.name}' has the 'termHasMeaningByContext' object property.")
            for related_ind in related_property:
                print(f" - Related individual: '{related_ind.name}'")
        else:
            print(f"Individual '{term_individual.name}' has 'termHasMeaningByContext' but no related individuals.")


In [ ]:
#updating termContentString based on context. If none is available, it will use WordNet definitions
from owlready2 import *
from nltk.corpus import wordnet

def extract_context_meaning(term_individual, nltk_context):
    context_meaning = ""
    connected_to = []

    try:
        if hasattr(term_individual, "termHasMeaningByContext"):
            related_property = term_individual.termHasMeaningByContext
            for related_ind in related_property:
                related_ind_name = related_ind.name if related_ind.name else ""
                connected_to.append(related_ind_name)

        if hasattr(term_individual, "termLexicon") and term_individual.termLexicon:
            term_lexicon = term_individual.termLexicon[0]  # Assuming a single value
            if term_lexicon in nltk_context.get("ContextDERHousehold", {}):
                context_meaning = nltk_context["ContextDERHousehold"][term_lexicon]
                source = "NLTK Context"
            else:
                synsets = wordnet.synsets(term_lexicon)
                if synsets:
                    context_meaning = synsets[0].definition()
                    source = "WordNet"

            if context_meaning:
                if not hasattr(term_individual, "termContentString"):
                    term_individual.termContentString = []
                term_individual.termContentString.append(context_meaning)
                return context_meaning, connected_to, term_lexicon, source
            else:
                return None, connected_to, term_lexicon, source
        else:
            return None, connected_to, None, None
    except AttributeError:
        return None, [], None, None

def process_individuals(onto, file_path):
    matching_individuals = [ind for ind in onto.individuals() if ind.name.startswith("ExchangedTerm")]

    for term_individual in matching_individuals:
        context_meaning, connected_to, term_lexicon, source = extract_context_meaning(term_individual, nltk_context)

        if context_meaning:
            with onto:
                sync_reasoner()  # Saving changes to the ontology
                onto.save(file_path)
                
            print(f"Individual: '{term_individual.name}'")
            print(f" - Object Property: 'termHasMeaningByContext'")
            print(f" - Connects to: {', '.join(connected_to) if connected_to else 'None'}")
            if term_lexicon:
                print(f" - Term Lexicon: {term_lexicon}")
            print(f" - Context Meaning: {context_meaning} (Source: {source})")
        else:
            print(" - No matching context found. NLTK WordNet used.")

def main():
    directory = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
    filename = 'oec-extracted.owl'
    file_path = os.path.join(directory, filename)

    try:
        onto = get_ontology("file://" + file_path).load()
        print("Ontology loaded successfully!")
        process_individuals(onto, file_path)
    except Exception as e:
        print("Error occurred during ontology loading:")
        print(e)

if __name__ == "__main__":
    nltk_context = {
        "ContextDERHousehold": {
            "Storage": "cells",
        }
    }
    main()


In [ ]:
from nltk.corpus import wordnet

# Function to get meanings of words using WordNet
def get_meanings(word):
    meanings = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            meanings.append(lemma.name())
    return list(set(meanings))

# Sample text
text = "NLTK is a powerful library for natural language processing."

# Tokenize the text into words
words = text.split()

# Dictionary to store words and their meanings
words_meanings = {}

# Get meanings for each word
for word in words:
    meanings = get_meanings(word)
    words_meanings[word] = meanings

# Print words and their meanings
for word, meanings in words_meanings.items():
    print(f"Word: {word}")
    if meanings:
        print(f"Meanings: {', '.join(meanings)}")
    else:
        print("No meanings found")
    print()
